In [2]:
import pandas
import openai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import slack_alarm
import requests

# slack 알람 사용법
# slack_alarm.send_notification("CSV 파일 처리")

In [3]:
# 데이터 확인
## 100개까지만 만든 dummy data
df = pd.read_csv("dummy_dataset.csv").drop(columns = 'index')
df_columns_sort = df[['text', 'user_data', 'detail', 'contents', 'ERD_data', 'relationships_data', 'API_specs_data']]
df_columns_sort.head()

,text,user_data,detail,contents,ERD_data,relationships_data,API_specs_data
0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹...","[{'projectName': 'StudyHub', 'projectTarget': ...","[{'requirementType': 'FUNCTIONAL1', 'content':...","{'project_summary': {'title': 'GQ 서비스', 'categ...","[{'name': 'User', 'attributes': [{'name': 'use...","[{'from': 'User', 'to': 'StudyGroup', 'type': ...","{'api_specification': {'openapi': '3.0.0', 'in..."
1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...,"[{'projectName': '기억의 다리', 'projectTarget': '치...","[{'requirementType': 'FUNCTIONAL1', 'content':...","{'project_summary': {'title': '기억박물관', 'catego...","[{'name': 'User', 'attributes': [{'name': 'use...","[{'from': 'User', 'to': 'Photo', 'type': 'one-...","{'api_specification': {'openapi': '3.0.0', 'in..."
2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로...",[{'projectName': 'EasyAccess Subway Navigator'...,"[{'requirementType': 'FUNCTIONAL1', 'content':...",{'project_summary': {'title': '보행 약자를 위한 지하철 길...,"[{'name': 'User', 'attributes': [{'name': 'use...","[{'from': 'User', 'to': 'Route', 'type': 'one-...","{'api_specification': {'openapi': '3.0.0', 'in..."
3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있...","[{'projectName': '이웃사촌', 'projectTarget': '1인 ...","[{'requirementType': 'FUNCTIONAL1', 'content':...","{'project_summary': {'title': '1인 가구 맞춤형 플랫폼',...","[{'name': 'User', 'attributes': [{'name': 'use...","[{'from': 'User', 'to': 'Post', 'type': 'one-t...","{'api_specification': {'openapi': '3.0.0', 'in..."
4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안...","[{'projectName': 'EmergencyCare Finder', 'proj...","[{'requirementType': 'FUNCTIONAL1', 'content':...",{'project_summary': {'title': '응급실 및 병원 찾기 웹앱'...,"[{'name': 'User', 'attributes': [{'name': 'use...","[{'from': 'User', 'to': 'EmergencyRoom', 'type...","{'api_specification': {'openapi': '3.0.0', 'in..."


In [4]:
df_top3 = df_columns_sort.head(1)
project_overview_lst = df_top3['user_data'].tolist()
existing_requirement_lst = df_top3['detail'].tolist()

content_lst = []
erd_lst = []
relationship_lst = []
API_spec_data_lst = []


project_overview = project_overview_lst[0]
existing_requirement = existing_requirement_lst[0]


In [ ]:
# 요청 데이터
data = {
    "project_overview": "string",
    "existing_requirements": "string",
    "additional_count": 5,
    "max_tokens": 4000,
    "temperature": 0.3,
    "model": "gpt-4o-mini"
  }

def requirements_data_create(data) :
   # API 호출
  response = requests.post(
      "http://127.0.0.1:8000/api/pja/requirements/generate",  # 로컬 서버 주소
      json=data,
      headers={"Content-Type": "application/json"}
  )

  # 응답 처리
  if response.status_code == 200:
      result = response.json()
      return result
  else:
      print(f"Error: {response.status_code}")
      print(response.text)


print(requirements_data_create(data)['requirements'])

[{'requirementType': 'FUNCTIONAL', 'content': '사용자는 시스템에 로그인할 때 2단계 인증을 설정할 수 있어야 하며, 이를 통해 보안을 강화할 수 있다.'}, {'requirementType': 'FUNCTIONAL', 'content': '시스템은 사용자가 생성한 데이터에 대해 자동 백업 기능을 제공해야 하며, 사용자는 백업 주기를 설정할 수 있어야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 1000명의 동시 사용자가 접속할 때도 평균 응답 시간이 2초 이내로 유지되어야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 데이터베이스 쿼리 처리 속도가 95%의 경우 1초 이내로 완료되어야 하며, 최대 처리량은 시간당 5000건 이상이어야 한다.'}]


In [22]:
# 요청 데이터
data = {
    "project_overview": project_overview,
    "requirements": existing_requirement,
    "max_tokens": 4000,
    "temperature": 0.3,
    "model": "ft:gpt-4o-mini-2024-07-18:test::BebIPMSD"
}

def json_data_create(data) :
  # API 호출
  response = requests.post(
      "http://127.0.0.1:8000/api/pja/json_text/generate",  # 로컬 서버 주소
      json=data,
      headers={"Content-Type": "application/json"}
  )

  # 응답 처리
  if response.status_code == 200:
      result = response.json()

  else:
      print(f"Error: {response.status_code}")
      print(response.text)
      # slack_alarm.send_notification(f"{1}번쨰 오류 발생")

  print(result)

In [23]:
json_data_create(data)

{'json': {'project_summary': {'title': 'StudyHub', 'category': '웹 플랫폼', 'target_users': ['대학생', '자격증 준비생'], 'core_features': ['스터디 그룹 생성 및 관리', '퀴즈 생성 및 참여 기능', '공지사항 및 과제 게시판'], 'technology_stack': ['Java Servlet', 'JSP', 'MySQL'], 'problem_solving': {'current_problem': '효율적인 학습 관리 및 사용자 간 소통 부족', 'solution_idea': '스터디 그룹 운영과 학습 내용을 복습할 수 있는 기능 제공', 'expected_benefits': ['학습 효과 극대화', '사용자 간 소통 강화', '학습 동기 부여']}}}, 'model': 'ft:gpt-4o-mini-2024-07-18:test::BebIPMSD', 'total_tokens': 1496, 'prompt_tokens': 1306, 'completion_tokens': 190}


In [ ]:
# df_top3.to_csv('dummy_data_utf-8.csv', encoding = 'utf-8', index = False)
# df_top3.to_csv('dummy_data_euc-kr.csv', encoding = 'euc-kr', index = False)